(sec:second-quant)=
# Second quantization

## Fock space

The way we described quantum mechanical states so-far contains a redundancy: it attributes individual particles (electrons) to a given state (spin-orbital), despite the fact that electrons are undistinguishable. This undistinguishability is included *a posteriori* by antisymmetrizing the state. This approach, called first quantization, while perfectly valid, can be cumbersome.

A more natural approach is to completely avoid identifying particles and instead simply ask "how many particles are present in a given quantum state?". To this end, we introduce the concept of occupation number vector, in which we list all possible states of the system (spin-orbitals) and their occupation. Note that for fermions, like electrons, the occupation can only be 0 or 1. We then define the Fock space as the abstract vector space of occupation vectors. Clearly there is a unique mapping between occupation number vectors and Slater Determinants as long as a spin-orbital basis is selected. Note however that the Fock space includes states with different total number of particles.

Let us for instance consider a Fock space consisting of three spin-orbitals: $\psi_1(\mathbf{x})$, $\psi_2(\mathbf{x})$, and $\psi_3(\mathbf{x})$. The Slater determinant for two electrons occupying $\psi_1(\mathbf{x})$ and $\psi_2(\mathbf{x})$ spin-orbitals $1/\sqrt{2} |\psi_1(\mathbf{x}) \psi_2(\mathbf{x}) |$ can be represented in Fock space as the occupation number vector $|\mathbf{k}\rangle = | 1 1 0 \rangle$. 

The Fock space is a linear vector space, and consequently has the following properties:

- two occupation number vectors $| \mathbf{k} \rangle$ and $| \mathbf{m} \rangle$ are orthogonal i.e., $\langle \mathbf{k} | \mathbf{m} \rangle = \delta_{\mathbf{k}, \mathbf{m}}$ 
-  occupation number vectors constitutes complete basis i.e. arbitrary vector $|\mathbf{g}\rangle$ can be expressed as linear combination of occupation number vectors

$$
| \mathbf{g} \rangle = \sum_\mathbf{k} c_\mathbf{k} | \mathbf{k} \rangle
$$

and outer product of occupation number vectors resolve to identity operator
$$
1 = \sum_\mathbf{k} | \mathbf{k} \rangle \langle  \mathbf{k} | 
$$

The Fock space consisting of $N$ spin orbitals, $F(N)$, can be partitioned into sub-spaces as 

$$
F(N) = F(0,N) \oplus F(1,N) \oplus ... \oplus F(N,N) 
$$

where $F(M, N)$ is the set of occupation number vectors with $M$ occupied spin-orbitals. The Fock subspace $F(M,N)$ can be viewed as an $M$-electron wave function expressed in occupation number vectors, as it contains all Slater determinant generated by distributing $M$ electrons in $N$ spin orbitals. Among the Fock subspaces, the $F(0, N)$ subspace corresponds to a completely unoccupied Slater determinant and is called a vacuum state $| vac \rangle$. The vacuum state has the following properties

$$
| vac \rangle = | 0_1, 0_2, ..., 0_N \rangle  \ \mathrm{and} \ \langle vac | vac \rangle = 1   
$$


## Creation and annihilation operators
Second quantization also introduces a very convenient way to manipulate states through creation and annihilation operators. As their name imply, those operators can be seen as creating and destroying electrons in specific spin-orbital. In terms of occupation vector manipulation, these actions are described

- creation operator $\hat{a}_p^\dagger$: 

$$
 \hat{a}_p^\dagger | \mathbf{k} \rangle = \delta_{k_p, 0} \prod_{i = 1}^{p} (-1)^{k_p} | k_1 k_2...1_p ...k_N \rangle 
$$

- annihilation operator $\hat{a}_p$: 

$$
 \hat{a}_p | \mathbf{k} \rangle = \delta_{k_p, 1} \prod_{i = 1}^{p} (-1)^{k_p} | k_1 k_2...0_p ...k_N \rangle 
$$


The $\delta_{k_p, 0}$ ensures that the creation operator cannot add an electron to a state that is already populated while $\delta_{k_p, 1}$ prevents the destruction of a non-existing electron. The phase factor $\prod_{i = 1}^{p} (-1)^{k_p}$ ensures the antisymmetry of the wavefunction.  However, the main strength of this formulation is that the phase factor is a direct consequence of the anti-commutator relationship of the creation and annihilation operators: 

\begin{eqnarray*}
\hat{a}_p^\dagger \hat{a}_q +  \hat{a}_q \hat{a}_p^\dagger &=& \delta_{p,q}  \ ,  \\ 
\hat{a}_p^\dagger \hat{a}_q^\dagger +  \hat{a}_q^\dagger \hat{a}_p^\dagger &=& 0 \ ,  \\
\hat{a}_p \hat{a}_q +  \hat{a}_q \hat{a}_p &=& 0 \ , 
\end{eqnarray*}

which are primary tools to manipulate occupation number vectors.

Having defined those operators, it is easy to see that any arbitrary state can be simply formed by applying a string of creation operators on the vacuum state. 

Similarly, the standard operators of quantum mechanics can be represented in second quantization using the creation and annihilation operators. Any operator $\hat{O}$ depending on the coordinates (both spatial and spin) $\mathbf{x}$ of a single electron can be expressed as a product of the integral of the operator in the chosen basis $O_{pq}= \int \psi_p^* (\mathbf{x}) \hat{O} (\mathbf{x}) \psi_q (\mathbf{x}) d\mathbf{x}$ and the excitation operator $\hat{a}_p^\dagger \hat{a}_q$:

$$
\hat{O} = \sum_{pq} O_{pq} \hat{a}_p^\dagger \hat{a}_q
$$

A similar formula can be used for two-electron operators. For example, the Hamiltonian of a molecular system can be written as:

$$
\hat H = \sum_{pq} h_{pq} \hat{a}_p^\dagger \hat{a}_q + \frac{1}{2} \sum_{pqrs} g_{pqrs} \hat{a}_p^\dagger \hat{a}_r^\dagger \hat{a}_s \hat{a}_q \ , 
$$

where $h_{pq}$ is the one-electron part of electronic Hamiltonian 

$$
 h_{pq} = \int \psi_p(\mathbf{x})^* \bigg \{ - \frac{1}{2} \hat{\nabla}^2 - \sum_K \frac{Z_k}{|\mathbf{r} - \mathbf{R}_K |} \bigg \} \psi_q(\mathbf{x}) d \mathbf{x} \ , 
$$

and $g_{pqrs}$ is the two-electron part of electronic Hamiltonian 

$$
g_{pqrs} = \int \int \psi_p(\mathbf{x})^* \psi_r(\mathbf{x}')^* \frac{1}{|\mathbf{r} - \mathbf{r}'|} \psi_q(\mathbf{x}) \psi_s(\mathbf{x}') d \mathbf{x} d \mathbf{x}' \ .  
$$

At the end, second quantization is a completely equivalent way to represent quantum mechanics, but by using a unified description of operator and states and reducing all the complications of the antisymmetry to simple commutation rules between the creation and annihilation operators, it is often a significantly more straightforward and compact way to derive various wave function models than the conventional approach based on the direct application of Slater-Condon rules to individual Slater determinants. For example, derivation of working equations for coupled-cluster singles and doubles wave function in second quantization becomes a single lecture affair, compared to the conventional way, which is several days undertaking. Thus, the second quantization is a valuable mathematical tool for working with correlated wave functions.


## Illustration

To illustrate the power and simplicity of this formulation, we can derive the energy of a Slater Determinant $| 0 \rangle$:

$$
\langle 0| \hat H | 0\rangle = \sum_{pq} h_{pq} \langle 0| \hat{a}_p^\dagger \hat{a}_q| 0\rangle + \frac{1}{2} \sum_{pqrs} g_{pqrs} \langle 0| \hat{a}_p^\dagger \hat{a}_r^\dagger \hat{a}_s \hat{a}_q | 0\rangle
$$

For this we only need to evaluate the density matrices $D_{pq} = \langle 0| \hat{a}_p^\dagger \hat{a}_q| 0\rangle$ and $d_{pqrs} = \langle 0| \hat{a}_p^\dagger \hat{a}_r^\dagger \hat{a}_s \hat{a}_q | 0\rangle$. 

For the one-particle density matrix $D_{pq}$ we immediately notice that the orbital $q$ has to be occupied for the element to be non-zero when the annihilation operator is acting on the right on $| 0\rangle$ (the same is true for $p$ when $\hat{a}_p^\dagger$ is acting on the left on $\langle 0|$). But then, we can apply the commutation rule to get:

$$
\langle 0| \hat{a}_p^\dagger \hat{a}_q | 0 \rangle = \delta_{p,q} \langle 0| 0 \rangle - \langle 0| \hat{a}_q \hat{a}_p^\dagger | 0 \rangle
$$

Now, $\hat{a}_q$ acts as a creation operator on the left on $\langle 0|$ and since we said $q$ was occupied, this last term gives zero. The only term left is thus $\delta_{p,q}$. The one-electron term thus gives:

$$
\sum_{pq} h_{pq} \langle 0| \hat{a}_p^\dagger \hat{a}_q| 0\rangle = \sum_{pq \in \mathrm{occupied}} h_{pq} \delta_{p,q} = \sum_{p \in \mathrm{occupied}} h_{pp}
$$

By using the indices $i, j, k, l$ to refer to occupied orbitals, we can simply write this term as $\sum_{i} h_{ii}$

We can now turn our attention the two-electron terms. Here both $q$ and $s$ have to be occupied. We will focus on $s$ which we will call $i$ and we will use the same trick of moving $\hat{a}_i$ to the left to generate a zero:

\begin{align*}
\hat{a}_p^\dagger \hat{a}_r^\dagger \hat{a}_i \hat{a}_q &= \delta_{r,i} \hat{a}_p^\dagger \hat{a}_q - \hat{a}_p^\dagger \hat{a}_i \hat{a}_r^\dagger \hat{a}_q\\
 &= \delta_{r,i} \hat{a}_p^\dagger \hat{a}_q - \delta_{p,i} \hat{a}_r^\dagger \hat{a}_q + \hat{a}_i \hat{a}_p^\dagger \hat{a}_r^\dagger \hat{a}_q
\end{align*}

The last term will now disappear and we are left with only terms depending on 2 creation or annihilation operators, which we have already solved earlier:

\begin{align*}
\frac{1}{2} \sum_{pqrs} g_{pqrs} \langle 0| \hat{a}_p^\dagger \hat{a}_r^\dagger \hat{a}_s \hat{a}_q | 0\rangle &= \frac{1}{2} \sum_{ipq} g_{pqii} \langle 0|\hat{a}_p^\dagger \hat{a}_q | 0\rangle - \sum_{iqr} g_{iqri} \langle 0| \hat{a}_r^\dagger \hat{a}_q | 0\rangle \\
& = \frac{1}{2} \sum_{ij} g_{jjii} - g_{ijji}
\end{align*}

Our final energy is thus:

$$
\langle 0| \hat H | 0\rangle = \sum_i h_{ii} + \frac{1}{2} \sum_{ij} g_{jjii} - g_{ijji}
$$


## Spin
